In [1]:
import pandas as pd
import numpy as np

In [2]:
df=pd.read_csv("/content/labeled_data.csv")

In [3]:
import re

def clean_tweet(text):
    # Remove HTML entities (e.g., &amp, &quot)
    text = re.sub(r'&[a-z]+;', '', text)

    # Remove mentions (@username)
    text = re.sub(r'@\w+', '', text)

    # Remove URLs
    text = re.sub(r'http\S+', '', text)

    # Remove numbers
    text = re.sub(r'\d+', '', text)

    # Remove special characters, hashtags, and extra whitespace
    text = re.sub(r'[^\w\s]', '', text)

    # Convert to lowercase
    text = text.lower()

    return text

# Apply to the dataset
df['cleaned_tweet'] = df['tweet'].apply(clean_tweet)


In [4]:
df

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet,cleaned_tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...,rt as a woman you shouldnt complain about cl...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,rt boy dats coldtyga dwn bad for cuffin dat ...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,rt dawg rt you ever fuck a bitch and she st...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,rt she look like a tranny
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,rt the shit you hear about me might be true ...
...,...,...,...,...,...,...,...,...
24778,25291,3,0,2,1,1,you's a muthaf***in lie &#8220;@LifeAsKing: @2...,yous a muthafin lie right his tl is trash ...
24779,25292,3,0,1,2,2,"you've gone and broke the wrong heart baby, an...",youve gone and broke the wrong heart baby and ...
24780,25294,3,0,3,0,1,young buck wanna eat!!.. dat nigguh like I ain...,young buck wanna eat dat nigguh like i aint fu...
24781,25295,6,0,6,0,1,youu got wild bitches tellin you lies,youu got wild bitches tellin you lies


In [5]:
!pip install transformers torch scikit-learn pandas


In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import AdamW


In [7]:
data = df[['cleaned_tweet', 'class']]  # Select relevant columns


In [8]:
data

,cleaned_tweet,class
0,rt as a woman you shouldnt complain about cl...,2
1,rt boy dats coldtyga dwn bad for cuffin dat ...,1
2,rt dawg rt you ever fuck a bitch and she st...,1
3,rt she look like a tranny,1
4,rt the shit you hear about me might be true ...,1
...,...,...
24778,yous a muthafin lie right his tl is trash ...,1
24779,youve gone and broke the wrong heart baby and ...,2
24780,young buck wanna eat dat nigguh like i aint fu...,1
24781,youu got wild bitches tellin you lies,1


In [9]:
# Load BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize the tweets
def tokenize_tweets(tweet):
    return tokenizer.encode_plus(
        tweet,
        add_special_tokens=True,
        max_length=128,  # Max token length
        padding='max_length',
        return_attention_mask=True,
        return_tensors='pt'
    )

# Apply the tokenizer to the cleaned tweets
data.loc[:, 'tokens'] = data['cleaned_tweet'].apply(lambda x: tokenize_tweets(x))


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
<ipython-input-9-207efeb4d9c2>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[:, 'tokens'] = data['cleaned_tweet'].apply(lambda x: tokenize_tweets(x))


In [10]:
# Define features and labels
X = data['tokens']
y = data['class']

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [11]:
# Ensure that X_train is a list of tokenized inputs
X_train = X_train.tolist()  # Convert X_train to list if it is a Pandas Series
X_test = X_test.tolist()    # Convert X_test to list if it is a Pandas Series

# Proceed with the rest of your code
train_inputs = torch.cat([X_train[i]['input_ids'] for i in range(len(X_train))], dim=0)
train_masks = torch.cat([X_train[i]['attention_mask'] for i in range(len(X_train))], dim=0)
train_labels = torch.tensor(y_train.values)

test_inputs = torch.cat([X_test[i]['input_ids'] for i in range(len(X_test))], dim=0)
test_masks = torch.cat([X_test[i]['attention_mask'] for i in range(len(X_test))], dim=0)
test_labels = torch.tensor(y_test.values)

# DataLoader setup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

batch_size = 16

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)


In [12]:
from transformers import AdamW

# Load the pre-trained BERT model for sequence classification
model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels=3,  # Adjust this based on the number of classes you have
    output_attentions=False,
    output_hidden_states=False
)

# Move model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
print("Device being used:", device)


# Define optimizer
optimizer = AdamW(model.parameters(), lr=1e-5)

# Training function
epochs = 3 # You can adjust the number of epochs

for epoch in range(epochs):
    print(f'Epoch {epoch + 1}/{epochs}')

    # Set model to training mode
    model.train()

    total_loss = 0

    for batch in train_dataloader:
        # Unpack the batch and load it onto the device (CPU/GPU)
        batch_input_ids = batch[0].to(device)
        batch_input_mask = batch[1].to(device)
        batch_labels = batch[2].to(device)

        # Clear previously calculated gradients
        model.zero_grad()

        # Forward pass
        outputs = model(batch_input_ids, token_type_ids=None, attention_mask=batch_input_mask, labels=batch_labels)

        # Get the loss
        loss = outputs.loss
        total_loss += loss.item()

        # Backward pass to calculate gradients
        loss.backward()

        # Update weights
        optimizer.step()

    avg_train_loss = total_loss / len(train_dataloader)
    print(f'Training loss: {avg_train_loss}')


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Device being used: cuda


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/3
Training loss: 0.3237496907777724
Epoch 2/3
Training loss: 0.21938609990080998
Epoch 3/3
Training loss: 0.1803181729587396


In [14]:
# Define the directory where you want to save the model
output_dir = '/content/model'

# Save the model's weights and configuration
model.save_pretrained(output_dir)

# Save the tokenizer if you are using it as well
tokenizer.save_pretrained(output_dir)

print(f"Model saved to {output_dir}")


Model saved to /content/model


In [15]:
import shutil

# Path to the saved model directory and output zip file
shutil.make_archive('model_save', 'zip', '/content/model')


'/content/model_save.zip'

In [16]:
from google.colab import files

# Download the zipped model
files.download('model_save.zip')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [2]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch

# Load the model and tokenizer from the saved directory
model_dir = 'C:/Users/Bikesh/Documents/BinisOffensiveModel/model_save'

# Load the tokenizer
tokenizer = BertTokenizer.from_pretrained(model_dir)

# Load the trained model
model = BertForSequenceClassification.from_pretrained(model_dir)

# Move model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Set model to evaluation mode
model.eval()


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [3]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch

# Sample input text (offensive language example)
input_text = "This is fucking stupid"

# Tokenize the input text
inputs = tokenizer.encode_plus(
    input_text,
    add_special_tokens=True,  # Add [CLS] and [SEP] tokens
    max_length=128,  # Max length to be consistent with training
    padding='max_length',  # Pad to max length
    truncation=True,  # Truncate if the input is too long
    return_tensors='pt'  # Return PyTorch tensors
)

# Move inputs to the same device as the model (CPU/GPU)
input_ids = inputs['input_ids'].to(device)
attention_mask = inputs['attention_mask'].to(device)

# Make predictions
with torch.no_grad():
    outputs = model(input_ids, attention_mask=attention_mask)

# Get the predicted label (0 for non-offensive, 1 for offensive, or other based on your classes)
predictions = torch.argmax(outputs.logits, dim=1)

# Output the prediction
if predictions == 1:
    print("Offensive language detected.")
else:
    print("No offensive language detected.")


Offensive language detected.


In [39]:
import torch

def detect_offensive_language(input_text, model, tokenizer, device):
    # Tokenize the input text
    inputs = tokenizer.encode_plus(
        input_text,
        add_special_tokens=True,  # Add [CLS] and [SEP] tokens
        max_length=128,  # Max length to be consistent with training
        padding='max_length',  # Pad to max length
        truncation=True,  # Truncate if the input is too long
        return_tensors='pt'  # Return PyTorch tensors
    )

    # Move inputs to the same device as the model (CPU/GPU)
    input_ids = inputs['input_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)

    # Make predictions
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)

    # Get the predicted label (0 for non-offensive, 1 for offensive, or other based on your classes)
    predictions = torch.argmax(outputs.logits, dim=1)

    # Return the prediction result
    if predictions.item() == 1:
        return "Offensive language detected."
    else:
        return "No offensive language detected."

# Example usage:
# Ensure you have loaded 'model', 'tokenizer', and set 'device' correctly beforehand
# result = detect_offensive_language("This is fucking stupid ", model, tokenizer, device)
# result2 = detect_offensive_language("Dumbass ", model, tokenizer, device)
# result3 = detect_offensive_language("Testing ", model, tokenizer, device)
result4=detect_offensive_language("dickhead", model, tokenizer, device)
# print(result)
# print(result2)
# print(result3)
print(result4)


Offensive language detected.


In [1]:
import gradio as gr
import torch
from transformers import BertTokenizer, BertForSequenceClassification

# Load model and tokenizer
model_dir = 'C:/Users/Bikesh/Documents/BinisOffensiveModel/model_save'  # Path to your saved model directory
tokenizer = BertTokenizer.from_pretrained(model_dir)
model = BertForSequenceClassification.from_pretrained(model_dir)

# Move model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Function for offensive language detection
def detect_offensive_language(input_text):
    # Tokenize the input text
    inputs = tokenizer.encode_plus(
        input_text,
        add_special_tokens=True,  # Add [CLS] and [SEP] tokens
        max_length=128,  # Max length to be consistent with training
        padding='max_length',  # Pad to max length
        truncation=True,  # Truncate if the input is too long
        return_tensors='pt'  # Return PyTorch tensors
    )

    # Move inputs to the same device as the model (CPU/GPU)
    input_ids = inputs['input_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)

    # Make predictions
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)

    # Get the predicted label (0 for non-offensive, 1 for offensive)
    predictions = torch.argmax(outputs.logits, dim=1)

    # Return the result
    if predictions.item() == 1:
        return "Offensive language detected."
    else:
        return "No offensive language detected."

# Create Gradio interface
iface = gr.Interface(
    fn=detect_offensive_language,  # Function to be called
    inputs="text",  # Text input for user
    outputs="text",  # Text output showing result
    title="Offensive Language Detection",
    description="Enter a sentence to detect whether it contains offensive language."
)

# Launch the interface
iface.launch()


C:\Users\Binisa\anaconda3\Lib\site-packages\transformers\utils\generic.py:260: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(
C:\Users\Binisa\anaconda3\Lib\site-packages\transformers\utils\generic.py:260: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(


HFValidationError: Repo id must be in the form 'repo_name' or 'namespace/repo_name': 'C:/Users/Bikesh/Documents/BinisOffensiveModel/model_save'. Use `repo_type` argument if needed.

In [2]:
!pip install gradio


In [3]:
!pip install transformers torch scikit-learn pandas
